## Unicité de la catégorie pour un cid

* trouver les cid qui apparaissent dans plusieurs catégories :
```
select * from (select cid, count(distinct categorie) from texte group by cid) as tmp where count > 1 limit 10
```
* ajouter le nombre de catégories
```
alter table texte add nb_categorie int;
update texte as t1 set nb_categorie = (select count(*) from texte as t2 where t1.cid = t2.cid);
```
* compter les anomalies (1772 = 886*2)
```
select count(*) from texte where nb_categorie > 1;
```
* modifier la validité des anomalies
```
alter table texte add valide_categorie_unique boolean;
update texte set valide_categorie_unique = (nb_categorie = 1);
update texte set valide = valide and valide_categorie_unique;
```

## Unicité des articles

* trouver les articles qui apparaissent plusieurs fois
```
select * from (select id_, count(*) from article group by id_) as tmp where count > 1 limit 10;
```
* ajouter le nombre d'occurrences
```
alter table article add nb_occurrence int;
update article as t1 set nb_occurrence = (select count(*) from article as t2 where t1.id_ = t2.id_);
```
* compter les anomalies (15774=7887*2)
```
select count(*) from article where nb_occurrence > 1;
```
* invalidider les articles
```
alter table article add valide_apparition_unique boolean;
update article set valide_apparition_unique = (nb_occurrence = 1);
update article set valide = valide and valide_apparition_unique;
```
* invalider les textes qui contiennent des articles invalides (1290, dont 22 qui ont une catégorie unique)
```
alter table texte add valide_articles_valides boolean;
update texte set valide_articles_valides = (
    select count(*) = 0 from article
    where
        texte.cid = article.cid and
        article.valide = False
);
update texte set valide = valide and valide_articles_valides;
```

96132 textes valides

## Unicité des section_ta

* trouver les section_ta qui apparaissent plusieurs fois
```
select * from (select id_, count(*) from section_ta group by id_) as tmp where count > 1 limit 10;
```
* ajouter le nombre d'occurrences
```
alter table section_ta add nb_occurrence int;
update section_ta as t1 set nb_occurrence = (select count(*) from section_ta as t2 where t1.id_ = t2.id_);
```
* compter les anomalies (2634=1317*2)
```
select count(*) from section_ta where nb_occurrence > 1;
```
* invalider les section_ta
```
alter table section_ta add valide_apparition_unique boolean;
update section_ta set valide_apparition_unique = (nb_occurrence = 1);
update section_ta set valide = valide and valide_apparition_unique;
```
* invalider les textes qui contiennent des section_ta invalides (252 tous déjà invalides)
```
alter table texte add valide_section_ta_valides boolean;
update texte set valide_section_ta_valides = (
    select count(*) = 0 from section_ta
    where
        texte.cid = section_ta.cid and
        section_ta.valide = False
);
update texte set valide = valide and valide_section_ta_valides;
```

96132 textes valides

## Vérifie que chaque cid a un exactement un id\_

* ajouter le nombre de version originale
```
alter table texte add nb_version_originale int;
update texte set nb_version_originale = (select count(*) from version where version.id_ = texte.cid);
```
* compter les anomalies (22 textes dont 8 précédement valides)
```
select count(*) from texte where nb_version_originale != 1;
```
* invalidider les textes
```
alter table texte add valide_version_originale_unique boolean;
update texte set valide_version_originale_unique = (nb_version_originale = 1);
update texte set valide = valide and valide_version_originale_unique;
```

96124 textes valides

## Comparaison struct et version

* détecter les struct sans version et vice versa (NB : categorie et cid ne doivent pas valoir NULL)

```
alter table struct add correspondance_version boolean;
update struct set correspondance_version = (
    select count(*) > 0
    from version
    where
        struct.base_origine = version.base_origine and
        struct.categorie = version.categorie and
        struct.cid = version.cid and
        struct.id_ = version.id_
);

alter table version add correspondance_struct boolean;
update version set correspondance_struct = (
    select count(*) > 0
    from struct
    where
        struct.base_origine = version.base_origine and
        struct.categorie = version.categorie and
        struct.cid = version.cid and
        struct.id_ = version.id_
);
```
* invalider les struct et version (0 struct, 464 version)
```
update struct set valide = valide and correspondance_version;
update version set valide = valide and correspondance_struct;
```
* invalider les textes qui contiennent des version ou struct invalides (xxx tous déjà invalides)

```
alter table texte add valide_struct_valides boolean;
update texte set valide_struct_valides = (
    select count(*) = 0 from struct
    where
        texte.cid = struct.cid and
        struct.valide = False
);
update texte set valide = valide and valide_struct_valides;

alter table texte add valide_version_valides boolean;
update texte set valide_version_valides = (
    select count(*) = 0 from version
    where
        texte.cid = version.cid and
        version.valide = False
);
update texte set valide = valide and valide_version_valides;

```
